In [1]:
import pandas as pd
import numpy as np
import ast
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

In [14]:
# Read the training and validation CSV files
train_df = pd.read_csv("examples/data/train_hc_dataset_small_20240915_044230.csv")
val_df = pd.read_csv("examples/data/val_hc_dataset_small_20240915_044230.csv")


# Preprocessing function
def preprocess_data(df):
    # Parse 'prob_diff_values' into list of floats
    df["prob_diff_sequence"] = df["prob_diff_values"].apply(ast.literal_eval)
    df["cosine_sequence"] = df["cosine_values"].apply(ast.literal_eval)

    # Encode 'verdict' into numerical labels
    le = LabelEncoder()
    df["verdict_label"] = le.fit_transform(df["verdict"])

    # Pad sequences to the maximum length
    max_prob_diff_len = df["prob_diff_sequence"].apply(len).max()
    max_cosine_len = df["cosine_sequence"].apply(len).max()

    def pad_sequence(seq, max_len):
        return seq + [0] * (max_len - len(seq))

    prob_diff_sequence_padded = np.array(
        df["prob_diff_sequence"].apply(lambda x: pad_sequence(x, max_prob_diff_len))
    )
    prob_diff_sequence_padded_df = pd.DataFrame(
        prob_diff_sequence_padded.tolist(),
        columns=[f"prob_diff_sequence_padded_{i}" for i in range(max_prob_diff_len)],
    )

    cosine_sequence_padded = np.array(
        df["cosine_sequence"].apply(lambda x: pad_sequence(x, max_cosine_len))
    )

    cosine_sequence_padded_df = pd.DataFrame(
        cosine_sequence_padded.tolist(),
        columns=[f"cosine_sequence_padded_{i}" for i in range(max_cosine_len)],
    )

    df = pd.concat([df, prob_diff_sequence_padded_df, cosine_sequence_padded_df], axis=1)

    # Compute statistical features
    df["prob_diff_mean"] = df["prob_diff_sequence"].apply(np.mean)
    df["prob_diff_std"] = df["prob_diff_sequence"].apply(np.std)
    df["prob_diff_max"] = df["prob_diff_sequence"].apply(np.max)
    df["prob_diff_min"] = df["prob_diff_sequence"].apply(np.min)
    df["prob_diff_median"] = df["prob_diff_sequence"].apply(np.median)
    df["prob_diff_q25"] = df["prob_diff_sequence"].apply(lambda x: np.percentile(x, 25))
    df["prob_diff_q75"] = df["prob_diff_sequence"].apply(lambda x: np.percentile(x, 75))

    df["cosine_mean"] = df["cosine_sequence"].apply(np.mean)
    df["cosine_std"] = df["cosine_sequence"].apply(np.std)
    df["cosine_max"] = df["cosine_sequence"].apply(np.max)
    df["cosine_min"] = df["cosine_sequence"].apply(np.min)
    df["cosine_median"] = df["cosine_sequence"].apply(np.median)
    df["cosine_q25"] = df["cosine_sequence"].apply(lambda x: np.percentile(x, 25))
    df["cosine_q75"] = df["cosine_sequence"].apply(lambda x: np.percentile(x, 75))

    return df


# Preprocess training and validation data
train_df = preprocess_data(train_df)
val_df = preprocess_data(val_df)

# Define feature columns
prob_diff_features = [
    "prob_diff_mean",
    "prob_diff_std",
    "prob_diff_max",
    "prob_diff_min",
    "prob_diff_median",
    "prob_diff_q25",
    "prob_diff_q75",
]

cosine_features = [
    "cosine_mean",
    "cosine_std",
    "cosine_max",
    "cosine_min",
    "cosine_median",
    "cosine_q25",
    "cosine_q75",
]

prob_diff_sequence_cols = [c for c in train_df.columns if "prob_diff_sequence_padded" in c]
cosine_sequence_cols = [c for c in train_df.columns if "cosine_sequence_padded" in c]

In [15]:
# Define datasets
class HallucinationDataset(Dataset):
    def __init__(self, df, X_columns, y_column):
        self.sequences = df[X_columns].values
        self.labels = df[y_column].values

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        sequence = torch.tensor(self.sequences[idx], dtype=torch.float32)
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        return sequence, label

    def sample(self, n):
        idx = np.random.choice(len(self), n, replace=False)
        return self.__getitem__(idx)


# Training loop
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=20):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for sequences, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(sequences)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # Validation
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for sequences, labels in val_loader:
                outputs = model(sequences)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        print(
            f"Epoch {epoch+1}/{epochs}, Train Loss: {running_loss/len(train_loader)}, Val Loss: {val_loss/len(val_loader)}, Val Accuracy: {100 * correct/total}%"
        )

In [16]:
# Prepare datasets and dataloaders
batch_size = 128

train_dataset = HallucinationDataset(train_df, prob_diff_sequence_cols, "verdict_label")
val_dataset = HallucinationDataset(val_df, prob_diff_sequence_cols, "verdict_label")

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

stats_train_dataset = HallucinationDataset(train_df, prob_diff_features, "verdict_label")
stats_val_dataset = HallucinationDataset(val_df, prob_diff_features, "verdict_label")

stats_train_loader = DataLoader(stats_train_dataset, batch_size=batch_size, shuffle=True)
stats_val_loader = DataLoader(stats_val_dataset, batch_size=batch_size, shuffle=False)

In [17]:
class LinearClassifier(nn.Module):
    def __init__(self, input_shape, output_shape):
        super(LinearClassifier, self).__init__()

        self.weights = nn.Linear(input_shape, output_shape)

    def forward(self, x):
        return self.weights(x)


X_sample, y_sample = stats_train_dataset.sample(1)

# Instantiate the model
model = LinearClassifier(input_shape=X_sample.shape[-1], output_shape=2)
print(model)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Train the model
train_model(model, stats_train_loader, stats_val_loader, criterion, optimizer, epochs=20)

LinearClassifier(
  (weights): Linear(in_features=7, out_features=2, bias=True)
)


/Users/jessicarumbelow/PIZZA/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Epoch 1/20, Train Loss: 0.6732943815223137, Val Loss: 0.651663503540096, Val Accuracy: 66.60824862717607%
Epoch 2/20, Train Loss: 0.6401324962853362, Val Loss: 0.6352753087655821, Val Accuracy: 67.06390933520271%
Epoch 3/20, Train Loss: 0.6282372298158801, Val Loss: 0.628185863370326, Val Accuracy: 66.64329945087043%
Epoch 4/20, Train Loss: 0.6224274630198663, Val Loss: 0.6247008349468459, Val Accuracy: 67.22747984577637%
Epoch 5/20, Train Loss: 0.6190712720539436, Val Loss: 0.6224232142540946, Val Accuracy: 67.34431592475757%
Epoch 6/20, Train Loss: 0.6162990508161389, Val Loss: 0.6207018903831938, Val Accuracy: 67.49620282743311%
Epoch 7/20, Train Loss: 0.6141786255549975, Val Loss: 0.6201649219242494, Val Accuracy: 67.81166024068233%
Epoch 8/20, Train Loss: 0.6125273568947428, Val Loss: 0.617969688639712, Val Accuracy: 67.58967169061806%
Epoch 9/20, Train Loss: 0.6114429092202576, Val Loss: 0.6171151982314551, Val Accuracy: 68.12711765393153%
Epoch 10/20, Train Loss: 0.6099601480070

In [18]:
class LinearClassifier(nn.Module):
    def __init__(self, input_shape, output_shape):
        super(LinearClassifier, self).__init__()

        self.weights = nn.Linear(input_shape, output_shape)

    def forward(self, x):
        return self.weights(x)


X_sample, y_sample = train_dataset.sample(1)

# Instantiate the model
model = LinearClassifier(input_shape=X_sample.shape[-1], output_shape=2)
print(model)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Train the model
train_model(model, train_loader, val_loader, criterion, optimizer, epochs=20)

LinearClassifier(
  (weights): Linear(in_features=296, out_features=2, bias=True)
)
Epoch 1/20, Train Loss: 0.6566719300245523, Val Loss: 0.6494654737301727, Val Accuracy: 64.35331230283911%
Epoch 2/20, Train Loss: 0.6435410907340152, Val Loss: 0.6456847653460147, Val Accuracy: 63.488725318378314%
Epoch 3/20, Train Loss: 0.6401916831859703, Val Loss: 0.6451910445049628, Val Accuracy: 61.52587919149433%
Epoch 4/20, Train Loss: 0.6381766074716789, Val Loss: 0.6438956923449217, Val Accuracy: 63.03306461035168%
Epoch 5/20, Train Loss: 0.6367162473723612, Val Loss: 0.6433325102969781, Val Accuracy: 62.76434162869494%
Epoch 6/20, Train Loss: 0.6358812482060281, Val Loss: 0.6426762191217337, Val Accuracy: 62.76434162869494%
Epoch 7/20, Train Loss: 0.635043342226053, Val Loss: 0.6440338127648653, Val Accuracy: 63.82754994742376%
Epoch 8/20, Train Loss: 0.634314519141365, Val Loss: 0.6422778986283203, Val Accuracy: 62.97464657086108%
Epoch 9/20, Train Loss: 0.6340894236073473, Val Loss: 0.64265

In [19]:
class SimpleNN(nn.Module):
    def __init__(self, input_shape, output_shape, breadth, depth, interpolate=True):
        super(SimpleNN, self).__init__()

        self.layers = []

        self.layers.append(nn.Linear(input_shape, breadth))
        self.layers.append(nn.ReLU())

        if interpolate:
            change_rate = int((breadth - output_shape) / max(1, depth))
            layer_sizes = [breadth - change_rate * i for i in range(depth)]

            for l in layer_sizes[:-1]:
                self.layers.append(nn.Linear(l, l - change_rate))
                self.layers.append(nn.ReLU())

            self.layers.append(nn.Linear(layer_sizes[-1], output_shape))

        else:
            for _ in range(depth - 1):
                self.layers.append(nn.Linear(breadth, breadth))
                self.layers.append(nn.ReLU())
            self.layers.append(nn.Linear(breadth, output_shape))

        self.layers = nn.Sequential(*self.layers)

    def forward(self, x):
        return self.layers(x)


X_sample, y_sample = train_dataset.sample(1)

# Instantiate the model
model = SimpleNN(input_shape=X_sample.shape[-1], output_shape=2, breadth=64, depth=2)
print(model)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Train the model
train_model(model, train_loader, val_loader, criterion, optimizer, epochs=20)

SimpleNN(
  (layers): Sequential(
    (0): Linear(in_features=296, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=65, bias=True)
    (3): ReLU()
    (4): Linear(in_features=65, out_features=2, bias=True)
  )
)
Epoch 1/20, Train Loss: 0.6113929350744501, Val Loss: 0.593511347895238, Val Accuracy: 69.57588503329828%
Epoch 2/20, Train Loss: 0.5734497975126357, Val Loss: 0.5830924259637719, Val Accuracy: 70.25353429138919%
Epoch 3/20, Train Loss: 0.5633064828205518, Val Loss: 0.582635097539247, Val Accuracy: 70.38205397826849%
Epoch 4/20, Train Loss: 0.5578267854426552, Val Loss: 0.5778400353086528, Val Accuracy: 71.30505900221989%
Epoch 5/20, Train Loss: 0.5487795044935824, Val Loss: 0.5925013367364655, Val Accuracy: 68.99170463839233%
Epoch 6/20, Train Loss: 0.546453017021965, Val Loss: 0.570870851069244, Val Accuracy: 71.316742610118%
Epoch 7/20, Train Loss: 0.5418298436336763, Val Loss: 0.5669747994684461, Val Accuracy: 71.29337539432177%
Epo

In [20]:
class SimpleConvNet(nn.Module):
    def __init__(self, input_shape, output_shape, breadth, depth):
        super(SimpleConvNet, self).__init__()

        layers = []
        in_channels = input_shape[0]
        ks = 3
        num_filters = breadth // (ks) ** 2

        # Add convolutional layers that maintain the input dimension
        for _ in range(depth):
            layers.append(nn.Conv1d(in_channels, num_filters, kernel_size=3, padding=1))
            layers.append(nn.ReLU())
            in_channels = num_filters

        self.conv_layers = nn.Sequential(*layers)

        # Calculate the size of the flattened output
        conv_output_shape = input_shape[1] * input_shape[2] * num_filters

        # Fully connected layer for classification
        self.fc = nn.Linear(conv_output_shape, output_shape)

    def forward(self, x):
        x = x.unsqueeze(1)
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)  # Flatten the output
        x = self.fc(x)
        return x


X_sample, y_sample = train_dataset.sample(1)
input_shape = X_sample.unsqueeze(1).shape
# Instantiate the model
model = SimpleConvNet(input_shape=input_shape, output_shape=2, breadth=64, depth=2)
print(model)

# Rest of the setup remains the same
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Train the model
train_model(model, train_loader, val_loader, criterion, optimizer, epochs=20)

SimpleConvNet(
  (conv_layers): Sequential(
    (0): Conv1d(1, 3, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): ReLU()
  )
  (fc): Linear(in_features=888, out_features=2, bias=True)
)
Epoch 1/20, Train Loss: 0.6475355827245589, Val Loss: 0.6315331699243233, Val Accuracy: 65.72029442691904%
Epoch 2/20, Train Loss: 0.6312549106552877, Val Loss: 0.6283989219523188, Val Accuracy: 66.10585348755696%
Epoch 3/20, Train Loss: 0.628067748485205, Val Loss: 0.6282000784108888, Val Accuracy: 66.60824862717607%
Epoch 4/20, Train Loss: 0.6257177230626216, Val Loss: 0.6252434538371527, Val Accuracy: 66.42131090080616%
Epoch 5/20, Train Loss: 0.6229434118250409, Val Loss: 0.6250542884887155, Val Accuracy: 65.95396658488141%
Epoch 6/20, Train Loss: 0.6210374681223104, Val Loss: 0.6265836106752282, Val Accuracy: 66.3161584297231%
Epoch 7/20, Train Loss: 0.6189656464838674, Val Loss: 0.6219079067457968, Val Accuracy: 66.3161584297231%
Epoch 8/20, Train Loss: 0.6172021056961092, Val Loss: 0.621704

In [ ]:
class SimpleRNN(nn.Module):
    def __init__(self, input_shape, output_shape, breadth, depth):
        super(SimpleRNN, self).__init__()
        self.rnn = nn.RNN(
            input_size=input_shape,
            hidden_size=breadth,
            num_layers=depth,
            batch_first=True,
            bidirectional=True,
        )
        self.fc = nn.Linear(breadth, output_shape)

    def forward(self, x):
        x = x.unsqueeze(1)
        _, h_n = self.rnn(x)
        out = self.fc(h_n[-1])
        return out


# Sample a batch from the dataset
X_sample, y_sample = train_dataset.sample(1)
input_shape = X_sample.shape[-1]  # Extract the number of features

# Instantiate the model
model = SimpleRNN(input_shape=input_shape, output_shape=2, breadth=64, depth=2)
print(model)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Train the model
train_model(model, train_loader, val_loader, criterion, optimizer, epochs=20)

SimpleRNN(
  (rnn): RNN(296, 64, num_layers=2, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=64, out_features=2, bias=True)
)
Epoch 1/20, Train Loss: 0.6388754970014351, Val Loss: 0.6359873498553661, Val Accuracy: 62.75265802079682%
Epoch 2/20, Train Loss: 0.6066626740115907, Val Loss: 0.6138586891231252, Val Accuracy: 67.26253066947073%
Epoch 3/20, Train Loss: 0.5937342344435499, Val Loss: 0.6046077875948664, Val Accuracy: 68.173852085524%
Epoch 4/20, Train Loss: 0.5843781340275711, Val Loss: 0.6009919881375868, Val Accuracy: 67.06390933520271%
Epoch 5/20, Train Loss: 0.5772432902340213, Val Loss: 0.588089759225276, Val Accuracy: 69.42399813062273%
Epoch 6/20, Train Loss: 0.5693036013406745, Val Loss: 0.582054913711192, Val Accuracy: 69.99649491763057%
Epoch 7/20, Train Loss: 0.5633276172717754, Val Loss: 0.5852905173799885, Val Accuracy: 69.84460801495501%
Epoch 8/20, Train Loss: 0.5638035929765824, Val Loss: 0.5800513820861702, Val Accuracy: 70.08996378081551%
Epo